# Rapport du projet : Prédiction du Prix des Voitures


## Objectif du Projet

##### Ce projet vise à développer un modèle de machine learning capable de prédire le prix d'une voiture en fonction de ses caractéristique. L'objectif est d'entraîner un modèle de régression et de l'intégrer dans une application Streamlit pour permettre aux utilisateurs d'obtenir une estimation du prix d'un véhicule.

## Explication du code

### Importation des Bibliothèques

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle

- **pandas** : Pour la manipulation des données, la lecture et l'écriture
- **numpy**  :  Pour les opérations numériques et la manipulation de tableaux
- **sklearn.model_selection** : Pour diviser les données en train/test
- **sklearn.metrics** : Pour mesurer la performance du modèle
- **sklearn.preprocessing** : Pour normaliser les données
- **sklearn.ensemble** : Pour créer un modèle de forêt de décision
- **sklearn.ensemble.RandomForestRegressor** : Modèle perfomant pour la régression.
- **pickel** : Pour sauvegarder et recharger le modèle



### Chargements des Données

In [19]:
data = pd.read_csv("scrap_price.csv")

##### Nous utilisons pd.read_csv pour charger les données.

### Nettoyage et Préparation des Données 

#### a) Suppression des Colonnes Non Pertinentes

In [20]:
colones_a_supprimer = ['ID','aspiration','doornumbers','enginelocation',"name","symboling","boreratio"]
data = data.drop(columns=colones_a_supprimer)


##### Pourquoi ? Ces colonnes n'influencent pas significativement le prix et sont donc supprimées.

#### b) Transformation des Variables Catégoriques

In [21]:
colonnes_categoriques = [ "fueltypes", "carbody", "drivewheels", "enginetype", "cylindernumber", "fuelsystem"]
data = pd.get_dummies(data, columns=colonnes_categoriques, drop_first=True)

##### Nous convertissons les variables textuelles en valeurs numériques via l'encodage One-Hot Encoding.

#### c) Gestion des Valeurs Manquantes

In [22]:
#Vérifier s’il y a des valeurs manquantes
data.isnull().sum()
data = data.dropna()

##### Nous utilisons data.dropna() pour supprimer les lignes avec des valeurs manquantes garantir la cohérence des données.

### Séparation des Données pour l'Entraînement

In [23]:
X= data.drop(columns=['price']) # Variables indépendantes (features)
y=data['price'] # La variable cible
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


##### Nous divisons les données en 80% d'entraînement et 20% de test. 

### Entraînement du Modèle

In [24]:
model =  RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train,y_train)

RandomForestRegressor(random_state=42)

##### Nous utilisons RandomForestRegressor car il gère bien les relations non linéaires et est moins sensibles aux valeurs aberrantes.

### Évaluation du Modèle 

In [25]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)  
mse = mean_squared_error(y_test, y_pred)  
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)  

print(" Évaluation du modèle :")
print(f" Mean Absolute Error (MAE) : {mae:.2f}")
print(f" Mean Squared Error (MSE) : {mse:.2f}")
print(f" Root Mean Squared Error (RMSE) : {rmse:.2f}")
print(f" R² Score : {r2:.4f}")  

 Évaluation du modèle :
 Mean Absolute Error (MAE) : 1228.42
 Mean Squared Error (MSE) : 3140478.13
 Root Mean Squared Error (RMSE) : 1772.14
 R² Score : 0.9602


#### Interprétation des métriques :

- **MAE (Mean Absolute Error)** : Erreur moyenne absolue.
- **MSE (Mean Squared Error)** : Erreur moyenne quadratique.
- **RMSE (Root Mean Squared Error)** : Erreur quadratique moyenne racine, plus interprétable.
- **R² Score** : Indique la qualité du modèle (plus proche de 1, mieux c'est).

##### R² = 0.9602 signifie que notre modèle explique 96% des variations du prix des voitures.

### Sauvegarde du Modèle

In [26]:
import pickle
with open("modele_voiture.pkl", "wb") as file:
    pickle.dump(model, file)
print(" Modèle sauvegardé avec succès !")


 Modèle sauvegardé avec succès !


## Déploiement avec Streamlit

##### Le modèle est ensuite utilisé dans une interface Streamlit permettant aux utilisateurs de saisir les caractéristiques du véhicule et d'obtenir une estimation du prix

### 1) Importation des Bibliothèques

In [27]:
import streamlit as st  # type: ignore # Framework pour créer l'interface utilisateur
import pandas as pd  # Manipulation des données
import pickle  # Chargement du modèle de Machine Learning
import numpy as np  # Calculs mathématiques

- **streamlit** : Permet de créer une interface web interactive.

- **pandas** : Pour gérer et organiser les données.

- **pickle** : Pour charger le modèle pré-entraîné.

- **numpy** : Pour les calculs mathématiques.



### 2) Chargement du Modèle de Machine Learning

In [28]:
@st.cache_resource  # Empêche de recharger le modèle à chaque interaction
def load_model():
    with open("modele_voiture.pkl", "rb") as file:
        return pickle.load(file)

model = load_model()# Charge le modèle une fois

- **@st.cache_resource** : Stocke le modèle en cache pour éviter de le recharger à chaque interaction.

- **pickle.load(file)** : Charge le modèle sauvegardé modele_voiture.pkl.

### 3) Définition des Colonnes Attendues par le Modèle

In [29]:
colonnes_model = [
    "wheelbase", "carlength", "carwidth", "carheight", "curbweight", "enginesize", "stroke",
    "compressionratio", "horsepower", "peakrpm", "citympg", "highwaympg",
    "fueltypes_gas", "carbody_hardtop", "carbody_hatchback", "carbody_sedan",
    "carbody_wagon", "drivewheels_fwd", "drivewheels_rwd", "enginetype_dohcv",
    "enginetype_l", "enginetype_ohc", "enginetype_ohcf", "enginetype_ohcv",
    "enginetype_rotor", "cylindernumber_five", "cylindernumber_four", 
    "cylindernumber_six", "cylindernumber_three", "cylindernumber_twelve",
    "cylindernumber_two", "fuelsystem_2bbl", "fuelsystem_4bbl", "fuelsystem_idi",
    "fuelsystem_mfi", "fuelsystem_mpfi", "fuelsystem_spdi", "fuelsystem_spfi"
]

- **Cette liste contient toutes les colonnes attendues par le modèle de Machine Learning.**

- **Les colonnes catégoriques (fueltypes, carbody, etc.) ont été transformées en variables binaires (One-Hot Encoding).**

- **Pour obtenir cette liste après l'entraînement du modèle, nous avons utiliser :** print(X_train.columns.tolist())

Cela affiche les colonnes utilisées pendant l'entraînement, qui doivent être les mêmes pour la prédiction.

### 4) Création de l'Interface Utilisateur

In [ ]:
st.title("🚗 Prédiction du Prix d'une Voiture")
st.write("Entrez les caractéristiques du véhicule et obtenez une estimation du prix.")

- **st.title()** : Définit un titre principal.

- **st.write()** : Ajoute une description textuelle.

### 5) Sélection des Options par l'Utilisateur

#### a)Type de carburant

In [ ]:
fueltypes = st.selectbox("Type de carburant ℹ️", ["Essence", "Diesel"],
                         help="Essence : Moins cher mais consomme plus.\nDiesel : Plus économique mais entretien plus coûteux.")

- **st.selectbox()** : Crée une liste déroulante permettant de choisir un carburant.

- **help** : Ajoute une infobulle explicative.

#### b) Type de carrosserie

In [ ]:
carbody = st.selectbox("Type de carrosserie ℹ️", ["Coupé", "Compacte", "Berline", "Break"],
                        help="🔹 **Coupé** : Toit rigide, 2 portes.\n🔹 **Compacte** : Petite, économique.\n🔹 **Berline** : 4 portes, confortable.\n🔹 **Break** : Plus grand espace de chargement.")

### c) Type de traction

In [ ]:
drivewheels = st.selectbox("Type de traction ℹ️", ["Traction avant", "Propulsion", "4x4"],
                            help="🔹 **Traction avant (FWD)** : Économique, stable.\n🔹 **Propulsion (RWD)** : Sportif, mais moins stable.\n🔹 **4x4 (AWD/4WD)** : Meilleure adhérence sur routes difficiles.")

### d) Type de Moteur

In [ ]:
enginetype = st.selectbox("Type de moteur ℹ️", ["DOHC", "Ligne", "OHC", "OHCF", "OHCV", "Rotatif"],
                           help="🔹 **DOHC** : Double arbre à cames, plus puissant.\n🔹 **Ligne (L)** : Cylindres alignés.\n🔹 **OHC / OHCF / OHCV** : Simples arbres à cames.\n🔹 **Rotatif** : Technologie spéciale (Mazda).")


### e) Nombre de Cylindres

In [ ]:
cylindernumber = st.selectbox("Nombre de cylindres ℹ️", ["2", "3", "4", "5", "6", "12"],
                               help="Plus de cylindres = Plus de puissance, mais consomme plus.\n🔹 **2 ou 3** : Petites voitures.\n🔹 **4** : Standard.\n🔹 **6** : Sportives & SUV.\n🔹 **12** : Supercars !")

### f) Système de carburant


In [ ]:
fuelsystem = st.selectbox("Système de carburant ℹ️", ["Injection MPFI", "Carburateur 2 corps", "Carburateur 4 corps", "Injection directe (IDI)"],
                           help="🔹 **MPFI** : Injection multi-points (moderne, précis).\n🔹 **Carburateur 2/4 corps** : Ancien système.\n🔹 **IDI** : Injection directe pour Diesel.")

### 6) Sélection des Caractéristiques Techniques avec des Sliders

In [ ]:
wheelbase = st.slider("Empattement (cm)", 80, 140, 100)
carlength = st.slider("Longueur du véhicule (cm)", 140, 220, 180)
carwidth = st.slider("Largeur du véhicule (cm)", 60, 90, 70)
carheight = st.slider("Hauteur du véhicule (cm)", 40, 80, 50)
curbweight = st.slider("Poids à vide (kg)", 500, 2500, 1500)
enginesize = st.slider("Taille du moteur (cm³)", 500, 5000, 2000)
stroke = st.slider("Course du piston", 2.0, 5.0, 3.5)
compressionratio = st.slider("Ratio de compression", 5.0, 25.0, 10.0)
horsepower = st.slider("Puissance (ch)", 50, 400, 150)
peakrpm = st.slider("Régime max (rpm)", 4000, 8000, 5500)
citympg = st.slider("Consommation en ville (mpg)", 5, 60, 30)
highwaympg = st.slider("Consommation sur autoroute (mpg)", 5, 60, 40)

- **st.slider() permet à l’utilisateur de choisir une valeur dans un intervalle défini.**

- **Les valeurs min/max ont été définies en analysant les données disponibles.**

- **Exemple :**

    - **wheelbase (empattement)** : 80 - 140 cm (gamme typique de voitures compactes à berlines).

    - **carlength (longueur)** : 140 - 220 cm (petites citadines à grandes berlines).

    - **carwidth (largeur)** : 60 - 90 cm (de très compact à large SUV).

### 7) Préparation des Données pour la Prédiction

In [35]:
input_data = {
    "wheelbase": wheelbase, "carlength": carlength, "carwidth": carwidth, 
    "carheight": carheight, "curbweight": curbweight, "enginesize": enginesize, 
    "stroke": stroke, "compressionratio": compressionratio, "horsepower": horsepower,
    "peakrpm": peakrpm, "citympg": citympg, "highwaympg": highwaympg
}

- **Stocke les valeurs entrées par l'utilisateur dans un dictionnaire.**

### 8) Conversion des Variables Catégoriques

In [42]:
input_data["fueltypes_gas"] = 1 if fueltypes == "Essence" else 0
input_data["carbody_" + carbody.lower()] = 1
input_data["drivewheels_" + drivewheels.lower()] = 1
input_data["enginetype_" + enginetype.lower()] = 1
input_data["cylindernumber_" + cylindernumber] = 1
input_data["fuelsystem_" + fuelsystem.lower().replace(" ", "_")] = 1

- **Convertit les valeurs textuelles en variables numériques binaires (0 ou 1).**
- **Assure que le modèle reçoit des données au bon format.**

### 9) Conversion en DataFrame et Réorganisation des Colonnes

In [ ]:
input_df = pd.DataFrame([input_data])

for col in colonnes_model:
    if col not in input_df.columns:
        input_df[col] = 0

input_df = input_df[colonnes_model]  # Réorganisation des colonnes du DataFrame pour qu'elles suivent exactement l'ordre spécifié dans colonnes_model.

### 10)  Bouton pour Lancer la Prédiction

In [ ]:
if st.button(" Prédire le Prix"):
    prix_estime = model.predict(input_df)
    st.success(f" Prix estimé : {prix_estime[0]:,.2f} $")


- **st.button()** : Affiche un bouton interactif.

- **model.predict(input_df)** : Utilise le modèle pour faire une prédiction du prix.

- **st.success()** : Affiche le résultat dans un cadre vert.

##### L'utilisateur peut entrer les valeurs, et l'application affiche le prix estimé en direct.

## Conclusion

##### Nous avons développé un modèle de machine learning performant pour prédire le prix des voitures et l'avons intégré dans une application interactive.